In [2]:
!pip install tensorflow==2.0.0

import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
print(keras.__version__)

2.0.0
2.2.4-tf


# Wide & Deep Neural Network Implementation

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [0]:
from tensorflow.keras import layers

input_ = layers.Input(shape = X_train.shape[1:])
hidden1 = layers.Dense(30, activation = 'relu')(input_)
hidden2 = layers.Dense(30, activation = 'relu')(hidden1)
concat = layers.Concatenate()([input_, hidden2])
output = layers.Dense(1)(concat)

model = keras.Model(inputs = [input_], outputs = [output])

# Multiple Input Wide & Deep
### Send a subset of the features through the wide path and a different subset (possibly overlapping) through the deep path using multiple inputs. In this example, we send features 0 to 4 through the wide path and 2 to 7 through deep path.

In [0]:
# input layers
input_A = layers.Input(shape = [5], name = "wide_input")
input_B = layers.Input(shape = [6], name = "deep_input")

# deep layers
hidden1 = layers.Dense(30, activation = 'relu')(input_B)
hidden2 = layers.Dense(30, activation = 'relu')(hidden1)

# concatenate wide & deep
concat = layers.concatenate([input_A, hidden2]) #functional interface

# ouput layer
output = layers.Dense(1, name = 'output')(concat)

# specify inputs and outputs to use
model = keras.Model(inputs = [input_A, input_B], outputs = [output])

In [12]:
model.compile(loss = 'mse', optimizer = keras.optimizers.SGD(lr = 1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))

mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 114us/sample - loss: 2.3497 - val_loss: 1.7249
Epoch 2/20
11610/11610 [==============================] - 1s 68us/sample - loss: 0.9537 - val_loss: 0.8093
Epoch 3/20
11610/11610 [==============================] - 1s 66us/sample - loss: 0.7434 - val_loss: 0.6944
Epoch 4/20
11610/11610 [==============================] - 1s 66us/sample - loss: 0.6801 - val_loss: 0.6420
Epoch 5/20
11610/11610 [==============================] - 1s 65us/sample - loss: 0.6394 - val_loss: 0.6085
Epoch 6/20
11610/11610 [==============================] - 1s 69us/sample - loss: 0.6093 - val_loss: 0.5769
Epoch 7/20
11610/11610 [==============================] - 1s 65us/sample - loss: 0.5851 - val_loss: 0.5546
Epoch 8/20
11610/11610 [==============================] - 1s 66us/sample - loss: 0.5656 - val_loss: 0.5353
Epoch 9/20
11610/11610 [==============================] - 1s 68us/sample - loss: 0.5493 - val_

# Multiple Output Wide & Deep

In [0]:
# input layers
input_A = layers.Input(shape = [5], name = "wide_input")
input_B = layers.Input(shape = [6], name = "deep_input")

# deep layers
hidden1 = layers.Dense(30, activation = 'relu')(input_B)
hidden2 = layers.Dense(30, activation = 'relu')(hidden1)

# concatenate wide & deep
concat = layers.concatenate([input_A, hidden2]) #functional interface

# ouput layer
output = layers.Dense(1, name = 'main_output')(concat)

# auxiliary layer
aux_output = layers.Dense(1, name = 'aux_output')(hidden2)

# specify inputs and outputs to use
model = keras.Model(inputs = [input_A, input_B], outputs = [output, aux_output])

In [0]:
# Set loss and loss weights for each output
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

In [15]:
history = model.fit(
    [X_train_A, X_train_B], [y_train, y_train], epochs=20,
    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 128us/sample - loss: 0.9628 - main_output_loss: 0.8299 - aux_output_loss: 2.1594 - val_loss: 0.5782 - val_main_output_loss: 0.5092 - val_aux_output_loss: 1.1993
Epoch 2/20
11610/11610 [==============================] - 1s 75us/sample - loss: 0.5561 - main_output_loss: 0.4953 - aux_output_loss: 1.1027 - val_loss: 0.5022 - val_main_output_loss: 0.4501 - val_aux_output_loss: 0.9703
Epoch 3/20
11610/11610 [==============================] - 1s 73us/sample - loss: 0.5001 - main_output_loss: 0.4541 - aux_output_loss: 0.9169 - val_loss: 0.4709 - val_main_output_loss: 0.4319 - val_aux_output_loss: 0.8217
Epoch 4/20
11610/11610 [==============================] - 1s 76us/sample - loss: 0.4723 - main_output_loss: 0.4362 - aux_output_loss: 0.7971 - val_loss: 0.4615 - val_main_output_loss: 0.4321 - val_aux_output_loss: 0.7258
Epoch 5/20
11610/11610 [==============================] - 1s 73us/

In [16]:
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

5160/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [0]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])